In [22]:
using Random
using Statistics
using LinearAlgebra
using Plots
using Polynomials
using JLD2
import Tensor_FixedSeedMC as TTMC

In [23]:
#import Pkg; Pkg.add("Polynomials")

In [24]:
# 標準正規乱数を生成する関数
function gen_sn(M, I; anti_paths=true, mo_match=true)
    Random.seed!(1234)
    if anti_paths
        sn = randn(M + 1, Int(I / 2))
        sn = hcat(sn, -sn)
    else
        sn = randn(M + 1, I)
    end
    if mo_match
        sn = (sn .- mean(sn)) ./ std(sn)
    end
    return sn
end

# アメリカン・プットオプションの価格を計算する関数
function price_american_put_option(K::Float64, S0::Float64, r::Float64, sigma::Float64, T::Float64; M::Int=50, I::Int=10000)
    dt = T / M                       # タイムステップの長さ
    df = exp(-r * dt)                # 割引ファクター
    S = zeros(M + 1, I)              # 資産価格パスの行列
    S[1, :] .= S0                    # 初期資産価格を設定
    sn = gen_sn(M, I)                # 標準正規乱数の生成

    # 資産価格パスのシミュレーション
    for t in 2:M+1
        S[t, :] = S[t - 1, :] .* exp.((r - 0.5 * sigma^2) * dt .+ sigma * sqrt(dt) .* sn[t, :])
    end

    h = max.(K .- S, 0.0)            # 即時行使価値
    V = copy(h)                      # オプション価値行列

    # Longstaff-Schwartzアルゴリズムの実装（逆順ループ）
    for t in M:-1:2
        discounted_V = V[t + 1, :] * df   # 次期のオプション価値を現在価値に割引

        # 回帰モデルのフィッティング（次数3の多項式）
        p = fit(S[t, :], discounted_V, 3)

        # 継続価値の推定
        C = p.(S[t, :])

        # 行使戦略の決定
        V[t, :] = ifelse.(C .> h[t, :], V[t + 1, :] * df, h[t, :])
    end

    C0 = df * mean(V[2, :])               # オプション価格の推定
    err = 1.96 * df * std(V[2, :]) / sqrt(I)     # オプション価格の標準誤差
    return C0, err
end

# 固定パラメータ
S0 = 100.0      # 初期資産価格
r = 0.05        # 無リスク金利
T = 1.0         # 満期（年）
M = 365         # タイムステップ数
I = 10000       # シミュレーションパス数

# 変化させるパラメータの範囲
S0_range = collect(90:0.3:119.7) # 200
sigma_range = collect(0.15:0.001:0.249) #200
# オプション価格を格納する行列
option_price = zeros(length(sigma_range), length(S0_range))


K = 100.0
sigma = sigma_range[1]
option_price, err = price_american_put_option(K, S0, r, sigma, T, M=M, I=I)

(4.01505063546102, 0.08615603218695884)

In [25]:
random_combinations = TTMC.generate_random_combinations(length(S0_range), 2, 100);

In [26]:
#=
res = []
errs = []
for i in S0_range
    for j in sigma_range
        option_price, err = price_american_put_option(K, i, r, j, T, M=M, I=I)
        push!(res, option_price)
        push!(errs, err)
    end
end
=#
price_matrix = Array{Float64}(undef, length(sigma_range), length(sigma_range))
err = Array{Float64}(undef, length(sigma_range), length(sigma_range))
for i in 1:length(S0_range)
    for j in 1:length(sigma_range)
        S0 = S0_range[i]
        sigma = sigma_range[j]
        #price_matrix[i, j] = asian_barrier_option(S0, K, B, T, sigma, mu, r, N_STEPS, N_PATHS)
        price_matrix[i, j], err[i, j] = price_american_put_option(K, S0, r, sigma, T, M=M, I=I)
    end
end

res = [price_matrix, err]


2-element Vector{Matrix{Float64}}:
 [10.18140318198178 10.178936389681878 … 12.74442947837249 12.779882955917083; 9.901496071020663 9.92909701103452 … 12.540662115754245 12.58773095744823; … ; 0.34209558143091084 0.35841936287147835 … 2.350010649940577 2.380281538221442; 0.3276097212152814 0.34026566151151877 … 2.304559883249757 2.328302995802493]
 [0.08055583043719607 0.08119543684142302 … 0.17486095776150104 0.1758764392511303; 0.0810899644181712 0.08362160294945277 … 0.1732129476054784 0.1743555738236753; … ; 0.026371419032937514 0.027196749304681127 … 0.09965600835816621 0.10058951594425894; 0.025558260233184034 0.026307977430554668 … 0.0983295058705615 0.09913445812366188]

In [27]:
JLD2.@save "american_S0_sigma_10_4.jld2" res